In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical

from keras.layers import Dense, Input
from keras.models import Sequential, Model

import os
import cv2
import glob
import matplotlib.pyplot as plt


In [ ]:
!pip install wandb
!pip install kaggle --upgrade
!pip install pandas numpy rasterio glob2

In [ ]:
import wandb

wandb.login()

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls -1ha kaggle.json

In [ ]:
!kaggle datasets download -d fpeccia/weed-detection-in-soybean-crops

In [ ]:
!unzip -q /content/weed-detection-in-soybean-crops.zip

In [ ]:
labels = ["broadleaf", "grass", "soil", "soybean"]

In [ ]:
X = []
Y = []
imgSize = (224, 224)
max_samples = 300
for i in labels:
  dataPath = "/content/dataset/" + i + "/"
  fileRead = glob.glob(dataPath + "*")[:max_samples]
  # print(fileRead)
  print(len(fileRead))

  for j in fileRead:
    image = cv2.imread(j)
    img = cv2.resize(image, imgSize)
    X.append(img)
    Y.append(i)



In [ ]:
X = np.array(X)
Y = np.array(Y)
  

In [ ]:
X.shape, Y.shape

In [ ]:
plt.imshow(X[0])

In [ ]:
oneHotY = []
for i in Y:
  oneHotY.append(labels.index(i))
Y = to_categorical(oneHotY)

In [ ]:
print(Y[0])
print(Y[1])
print(Y[2])
print(Y.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [ ]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D

# eliminate top layer
base_model = tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# add new classifier layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
num_classes = Y.shape[-1] 
output_layer = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output_layer)


for layer in base_model.layers:
    layer.trainable = True


optimizer = tf.keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# normalization
X_train = X_train / 255.0
X_test = X_test / 255.0

# train model
train_history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10, batch_size=32)

